In [ ]:
from sklearn.neighbors import NearestNeighbors

# Load previous SVD (dim=100)
svd = joblib.load("svd_100.joblib")

# Build a NearestNeighbors index (on reduced space)
nn = NearestNeighbors(n_neighbors=5, algorithm="auto", metric="cosine")

# --- Fit index in streaming (collect reduced rows in one array)
reduced_vectors = []
batch = []
for row in data.toLocalIterator():
    batch.append(row)
    if len(batch) >= BATCH:
        Xb = to_csr(batch)
        Xr = svd.transform(Xb)
        reduced_vectors.append(Xr)
        batch.clear()
if batch:
    Xb = to_csr(batch); Xr = svd.transform(Xb); reduced_vectors.append(Xr)

X_all = np.vstack(reduced_vectors)
nn.fit(X_all)

# Save index
joblib.dump({"svd": svd, "nn": nn, "X_all": X_all}, "neighbors.joblib")

# --- Example: query nearest neighbors for company i=123
distances, indices = nn.kneighbors(X_all[123].reshape(1,-1))
print("Nearest neighbor indices:", indices, "distances:", distances)